<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
# Instala la librería beautifulsoup4 para analizar documentos HTML y XML.
!pip3 install beautifulsoup4
# Instala la librería requests para hacer solicitudes HTTP.
!pip3 install requests

In [ ]:
# Importa el módulo sys, que proporciona acceso a variables y funciones del sistema Python.
import sys

# Importa la librería requests para realizar solicitudes HTTP.
import requests
# Importa BeautifulSoup de bs4 para analizar HTML y XML.
from bs4 import BeautifulSoup
# Importa el módulo re para operaciones con expresiones regulares.
import re
# Importa unicodedata para manejar propiedades de caracteres Unicode.
import unicodedata
# Importa pandas para manipulación y análisis de datos, dándole el alias 'pd'.
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [ ]:
# Define una función para extraer la fecha y hora de las celdas de una tabla HTML.
def date_time(table_cells):
    """
    Esta función devuelve la fecha y hora de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla que extrae filas adicionales
    """
    # Extrae el texto de las celdas, elimina espacios en blanco y toma los dos primeros elementos (fecha y hora).
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

# Define una función para extraer la versión del propulsor de las celdas de una tabla HTML.
def booster_version(table_cells):
    """
    Esta función devuelve la versión del propulsor de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla que extrae filas adicionales
    """
    # Concatena los elementos de texto de las celdas, filtrando los impares, para obtener la versión del propulsor.
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

# Define una función para extraer el estado de aterrizaje de las celdas de una tabla HTML.
def landing_status(table_cells):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla que extrae filas adicionales
    """
    # Retorna el primer elemento de texto encontrado dentro de la celda, que representa el estado de aterrizaje.
    out=[i for i in table_cells.strings][0]
    return out


# Define una función para extraer la masa de la carga útil de las celdas de una tabla HTML.
def get_mass(table_cells):
    # Normaliza la cadena de texto, elimina caracteres especiales y espacios en blanco.
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    # Verifica si la cadena de masa no está vacía.
    if mass:
        # Busca la posición de 'kg' en la cadena (el resultado de find() no se usa directamente aquí).
        mass.find("kg")
        # Extrae la subcadena desde el inicio hasta 'kg' inclusive, más los dos caracteres siguientes si existen.
        new_mass=mass[0:mass.find("kg")+2]
    # Si la masa está vacía, establece new_mass en 0.
    else:
        new_mass=0
    # Retorna la masa extraída o 0 si no se encontró.
    return new_mass


# Define una función para extraer el nombre de una columna de un elemento de encabezado de tabla HTML.
def extract_column_from_header(row):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla que extrae filas adicionales
    """
    # Si la celda contiene una etiqueta <br>, la elimina del árbol HTML.
    if (row.br):
        row.br.extract()
    # Si la celda contiene una etiqueta <a>, la elimina del árbol HTML.
    if row.a:
        row.a.extract()
    # Si la celda contiene una etiqueta <sup>, la elimina del árbol HTML.
    if row.sup:
        row.sup.extract()

    # Une los contenidos restantes de la celda para formar el nombre de la columna, separados por espacios.
    colunm_name = ' '.join(row.contents)

    # Filtra los nombres que son solo dígitos o están vacíos.
    # Si el nombre de la columna, después de eliminar espacios, no es solo un dígito.
    if not(colunm_name.strip().isdigit()):
        # Elimina los espacios en blanco al principio y al final del nombre de la columna.
        colunm_name = colunm_name.strip()
        # Retorna el nombre de la columna limpio.
        return colunm_name

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [ ]:

# Define la URL estática de la página de Wikipedia a raspar, que es una versión específica de la página.
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Define un diccionario de cabeceras HTTP para simular una solicitud de navegador web.
headers = {
    # Establece la cabecera User-Agent para identificarse como un navegador Chrome en Windows.
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [ ]:
# Se usará el método requests.get() con la URL estática y las cabeceras proporcionadas.
# El objeto de respuesta se asignará a una variable.
response = requests.get(static_url, headers=headers)

Create a `BeautifulSoup` object from the HTML `response`


In [ ]:
# Se utilizará BeautifulSoup() para crear un objeto BeautifulSoup a partir del contenido de texto de una respuesta HTTP.
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly


In [ ]:
# Se utilizará el atributo soup.title para imprimir el título de la página HTML.
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [ ]:
# Se utilizará la función find_all en el objeto BeautifulSoup para buscar todos los elementos de tipo `table`.
# El resultado de la búsqueda se asignará a una lista llamada `html_tables`.
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [ ]:
# Se imprime la tercera tabla (índice 2) de la lista `html_tables` para verificar su contenido.
first_launch_table = html_tables[2]
# Imprime el contenido HTML de la primera tabla de lanzamiento.
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [ ]:
# Inicializa una lista vacía para almacenar los nombres de las columnas.
column_names = []

# Aplica la función find_all() con el elemento `th` (encabezado de tabla) en `first_launch_table`.
# Itera sobre cada elemento `th` y aplica la función `extract_column_from_header()` proporcionada para obtener un nombre de columna.
# Agrega el nombre de columna no vacío (si el nombre no es None y tiene una longitud mayor que 0) a una lista llamada `column_names`.
column_header = first_launch_table.find_all('th')
for th in column_header:
    column_name = extract_column_from_header(th)
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [ ]:
# Imprime la lista `column_names` para mostrar los nombres de las columnas extraídas.
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
# Inicializa un diccionario llamado `launch_dict` con claves obtenidas de `column_names`, y cada valor inicialmente como `None`.
launch_dict= dict.fromkeys(column_names)

# Elimina una columna irrelevante del diccionario `launch_dict`.
del launch_dict['Date and time ( )']

# Inicializa las listas vacías para cada clave relevante en `launch_dict`.
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Agrega algunas nuevas columnas al diccionario `launch_dict` con listas vacías.
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
# Inicializa una variable para contar las filas extraídas.
extracted_row = 0
# Itera sobre cada tabla HTML con las clases 'wikitable plainrowheaders collapsible' encontrada en el objeto 'soup'.
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # Itera sobre cada fila ('tr') dentro de la tabla actual.
    for rows in table.find_all("tr"):
        # Comprueba si la fila tiene un encabezado de tabla ('th').
        if rows.th:
            # Comprueba si el encabezado de tabla tiene contenido de texto.
            if rows.th.string:
                # Extrae el número de vuelo, eliminando espacios en blanco.
                flight_number=rows.th.string.strip()
                # Verifica si el número de vuelo es completamente numérico.
                flag=flight_number.isdigit()
        # Si la fila no tiene un encabezado 'th' o no tiene texto, establece la bandera en falso.
        else:
            flag=False
        # Encuentra todas las celdas de datos ('td') dentro de la fila actual.
        row=rows.find_all('td')
        # Procede si el 'flag' es verdadero, indicando un número de vuelo válido.
        if flag:
            # Incrementa el contador de filas extraídas.
            extracted_row += 1
            # Valor del número de vuelo.
            # TODO: Añadir el número de vuelo al diccionario 'launch_dict' con la clave 'Flight No.'.
            print(flight_number)
            launch_dict['Flight No.'].append(flight_number)
            # Extrae la fecha y hora usando la función auxiliar 'date_time'.
            datatimelist=date_time(row[0])

            # Valor de la fecha.
            # TODO: Añadir la fecha al diccionario 'launch_dict' con la clave 'Date'.
            # Extrae la fecha de la lista y elimina la coma.
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)

            # Valor de la hora.
            # TODO: Añadir la hora al diccionario 'launch_dict' con la clave 'Time'.
            # Extrae la hora de la lista.
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            print(time)

            # Versión del propulsor.
            # TODO: Añadir la versión del propulsor ('bv') al diccionario 'launch_dict' con la clave 'Version Booster'.
            # Extrae la versión del propulsor usando la función auxiliar 'booster_version'.
            bv=booster_version(row[1])
            # Si 'bv' está vacío, intenta extraerlo de un enlace ('a') dentro de la celda.
            if not(bv):
                bv=row[1].a.string
            # Imprime la versión del propulsor para depuración.
            launch_dict['Version Booster'].append(bv)
            print(bv)

            # Sitio de lanzamiento.
            # TODO: Añadir el sitio de lanzamiento ('launch_site') al diccionario 'launch_dict' con la clave 'Launch Site'.
            # Extrae el sitio de lanzamiento del texto del enlace en la tercera celda de datos.
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)

            # Carga útil (Payload).
            # TODO: Añadir la carga útil ('payload') al diccionario 'launch_dict' con la clave 'Payload'.
            # Extrae la carga útil del texto del enlace en la cuarta celda de datos.
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(payload)

            # Masa de la carga útil.
            # TODO: Añadir la masa de la carga útil ('payload_mass') al diccionario 'launch_dict' con la clave 'Payload mass'.
            # Extrae la masa de la carga útil usando la función auxiliar 'get_mass'.
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload)

            # Órbita.
            # TODO: Añadir la órbita ('orbit') al diccionario 'launch_dict' con la clave 'Orbit'.
            # Extrae la órbita del texto del enlace en la sexta celda de datos.
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(orbit)

            # Cliente.
            # TODO: Añadir el cliente ('customer') al diccionario 'launch_dict' con la clave 'Customer'.
            # Extrae el cliente del texto del enlace en la séptima celda de datos.
            target_cell = row[6]
            if target_cell.a:
                # Si existe el enlace <a>, extraemos su texto
                customer = target_cell.a.string
            else:
                # Si no hay enlace, extraemos el texto directamente de la celda
                # .strip() limpia espacios en blanco o saltos de línea molestos
                customer = target_cell.get_text().strip()

            launch_dict['Customer'].append(customer)
            print(customer)

            # Resultado del lanzamiento.
            # TODO: Añadir el resultado del lanzamiento ('launch_outcome') al diccionario 'launch_dict' con la clave 'Launch outcome'.
            # Extrae el resultado del lanzamiento del primer string en la octava celda de datos.
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(launch_outcome)

            # Aterrizaje del propulsor.
            # TODO: Añadir el estado de aterrizaje del propulsor ('booster_landing') al diccionario 'launch_dict' con la clave 'Booster landing'.
            # Extrae el estado de aterrizaje del propulsor usando la función auxiliar 'landing_status'.
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)

1
4 June 2010
18:45
F9 v1.07B0003.18
CCAFS
Dragon Spacecraft Qualification Unit
Dragon Spacecraft Qualification Unit
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.07B0004.18
CCAFS
Dragon
Dragon
LEO
NASA
Success
Failure
3
22 May 2012
07:44
F9 v1.07B0005.18
CCAFS
Dragon
Dragon
LEO
NASA
Success
No attempt

4
8 October 2012
00:35
F9 v1.07B0006.18
CCAFS
SpaceX CRS-1
SpaceX CRS-1
LEO
NASA
Success

No attempt
5
1 March 2013
15:10
F9 v1.07B0007.18
CCAFS
SpaceX CRS-2
SpaceX CRS-2
LEO
NASA
Success

No attempt

6
29 September 2013
16:00
F9 v1.17B10038
VAFB
CASSIOPE
CASSIOPE
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
SES-8
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
Thaicom 6
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
SpaceX CRS-3
LEO
NASA
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
Orbcomm-OG2
LEO
Orbcomm
Success
Controlled
11
5 August 2014

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [ ]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
